Create a view in the first stage.
The view has already created in the database.
The following cell is the SQL for Station_Emissions View

In [2]:
#CREATE VIEW Station_Emissions AS
#SELECT
#    rs.StationID,
#    r.RouteID,
#    SUM(
#        CASE 
#            WHEN pc.PassengerCount > 0 AND rpc.TotalPassengerCount > 0 AND c.TotalCarbonEmissions_kg > 0
#            THEN c.TotalCarbonEmissions_kg * pc.PassengerCount / rpc.TotalPassengerCount
#            ELSE 0
#        END
#    ) AS StationCarbonEmissions
#FROM
#    RouteStation rs
#JOIN
#    Route r ON rs.RouteID = r.RouteID
#JOIN
#    Passenger_Counts pc ON rs.StationID = pc.StationID
#JOIN
#    Carbon_Emission c ON r.EmissionID = c.EmissionID
#JOIN
#    (
#        SELECT
#            rs.RouteID,
#            SUM(pc.PassengerCount) AS TotalPassengerCount
#        FROM
#            RouteStation rs
#        JOIN
#            Passenger_Counts pc ON rs.StationID = pc.StationID
#        WHERE pc.PassengerCount > 0
#        GROUP BY
#            rs.RouteID
#    ) rpc ON r.RouteID = rpc.RouteID
#GROUP BY
#    rs.StationID, r.RouteID

In [2]:
import sqlite3
import pandas as pd

db_path = 'C:/Users/10713/Desktop/LondonTransportation.db' 
conn = sqlite3.connect(db_path)

query = """
SELECT 
    s.StationID,
    s.Latitude,
    s.Longitude,
    SUM(se.StationCarbonEmissions) AS TotalStationEmissions
FROM 
    Station s
JOIN 
    Station_Emissions se ON s.StationID = se.StationID
GROUP BY 
    s.StationID, s.Latitude, s.Longitude;
"""
df = pd.read_sql_query(query, conn)

output_path = "C:/Users/10713/Desktop/station_emissions_with_location.csv"
df.to_csv(output_path, index=False)

conn.close()

output_path


'C:/Users/10713/Desktop/station_emissions_with_location.csv'

In [3]:
pip install folium pandas



  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/fc/ab/d1f47c48a14e17cd487c8b467b573291fae75477b067241407e7889a3692/folium-0.19.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/f8/9d/91cddd38bd00170aad1a4b198c47b4ed716be45c234e09b835af41f4e717/branca-0.8.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/110.5 kB ? eta -:--:--
   ---------------------------------------- 110.5/110.5 kB 6.3 MB/s eta 0:00:00


In [22]:
import pandas as pd
import folium
from folium.plugins import HeatMap

data = pd.read_csv("station_emissions_with_location.csv")

map_center = [data['Latitude'].mean(), data['Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=12)

heat_data = [
    [row['Latitude'], row['Longitude'], row['TotalStationEmissions']] 
    for index, row in data.iterrows()
]

HeatMap(heat_data, radius=15, blur=10).add_to(m)

m.save("station_emissions_heatmap.html")
print("Heatmap saved as station_emissions_heatmap.html")


Heatmap saved as station_emissions_heatmap.html
